In [1]:
!pip install datasets rouge opencv-python-headless evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=c0a0321fe5e7b65f9edb94f6f2e29ea7e5a927cc32f06e6f88ef2f7ccaa727bc
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's depen

In [2]:
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset


In [3]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [4]:
# ds = load_dataset("microsoft/MeetingBank-QA-Summary")
# df = pd.read_parquet("hf://datasets/microsoft/MeetingBank-QA-Summary/data/test-00000-of-00001.parquet")

In [5]:
df = pd.read_parquet("/content/0000.parquet")
df.drop(["idx","QA_pairs","summary"],axis=1,inplace=True)

In [6]:
df.head()

,prompt,gpt4_summary
0,"The report of the Civil Rights, Utilities, Eco...","The Civil Rights, Utilities, Economic Developm..."
1,"Madam Court, could you please read docket 1239...",The Committee on Government Operations discuss...
2,"Item 15, report from City Manager Recommendati...",The City Manager recommended the adoption of t...
3,"Item five, proposed ordinance 2016 0392. This ...",The meeting discussed proposed ordinance 2016 ...
4,Very good. Any comments? Those in favor of con...,The meeting discussed the confirmation of an a...


In [7]:
def tokenize_function(examples):
    inputs = tokenizer(examples["prompt"], truncation=True, max_length=1024, padding="max_length")
    targets = tokenizer(examples["gpt4_summary"], truncation=True, max_length=512, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

dataset = Dataset.from_pandas(df)


sample_text = df.iloc[0]["prompt"]
sample_description = df.iloc[0]["gpt4_summary"]
print("Sample Input:", sample_text)
print("Sample Target:", sample_description)

tokenized_sample = tokenizer(sample_text, truncation=True, max_length=128, padding="max_length")
print("Tokenized Input:", tokenized_sample)

tokenized_target = tokenizer(sample_description, truncation=True, max_length=128, padding="max_length")
print("Tokenized Target:", tokenized_target)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["prompt", "gpt4_summary"])

Sample Input: The report of the Civil Rights, Utilities, Economic Development and Arts Committee Agenda Item three Resolution 31669 Encouraging as a best practice the use of an individualized tenant assessment using the Fair Housing Act's discriminatory effect standards to avoid Fair Housing Act violations when criminal history is used as a screening criterion in the Landlord Screening Process, Committee recommends that the resolution be adopted as amended grade. I move to amend Resolution 31669 by substituting D four for version D three, which includes a new attachment. A And I understand Councilmember Bagshaw also has an amendment, but let's first, if we could, let me just go through the changes to the resolution since the last committee meeting. The changes are found in two recitals, as well as sections one and five are the primary changes. We added a recital that again lifts up the HUD guidance to show that a criminal history screening policy is next must serve a substantial, legit

Map:   0%|          | 0/862 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 862
})

In [9]:
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
valid_dataset = train_test_split["test"]
print(f"Train Size: {len(train_test_split['train'])}, Test Size: {len(train_test_split['test'])}")

Train Size: 775, Test Size: 87


In [10]:
from evaluate import load
rouge = load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {k: round(v * 100, 2) for k, v in result.items()}

In [11]:
#training arguments
training_args = TrainingArguments(
    output_dir="./BART-finetuned-meetings",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=5e-5,
    logging_steps=100,
    gradient_accumulation_steps=4,
    dataloader_num_workers=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer,
)
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shalavyaagrawal (shalavyaagrawal-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,1.749029
2,No log,0.545974
3,3.220300,0.467662
4,0.477500,0.452154


TrainOutput(global_step=240, training_loss=1.6097103595733642, metrics={'train_runtime': 295.2073, 'train_samples_per_second': 13.126, 'train_steps_per_second': 0.813, 'total_flos': 2319437487144960.0, 'train_loss': 1.6097103595733642, 'epoch': 4.907216494845361})

In [12]:
trainer.evaluate()

{'eval_loss': 0.4521537721157074,
 'eval_runtime': 2.1625,
 'eval_samples_per_second': 40.231,
 'eval_steps_per_second': 10.173,
 'epoch': 4.907216494845361}

In [13]:
trainer.save_model("./saved-bart-meeting-summary")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
